In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import re

In [ ]:
def get_soup(url):
    req = requests.get(url)      #html태그와 content내용을 Response 객체로 반환
    html = req.text                 
    soup = BeautifulSoup(html, 'html.parser')
    
    return soup
    
    
def get_df_data_on_site(soup):
    col = []
    data = []
    box = soup.select('.tinytable > tbody tr')   #n 개의 div
    title = soup.select('.tinytable > thead th')   #n 개의 div
    
    for txt in title:
        col.append(txt.text)
        
    for row in box:
        row_data = []
        boxbox = row.select('td')
        
        for txt in boxbox:
            row_data.append(txt.text)
            
        data.append(row_data)
        
    return col, data


def make_refine_df(col, data):
    df = pd.DataFrame(data, columns=col)
    drop_col = ['X', '1d', '1w', '1m', '6m']
    df.drop(drop_col, axis=1, inplace=True)
    
    return df




In [ ]:
if __name__ == "__main__":
    url = 'http://openinsider.com/latest-cluster-buys'
    soup = get_soup(url)
    col, data = get_df_data_on_site(soup)
    df = make_refine_df(col, data)

df

,Filing Date,Trade Date,Ticker,Company Name,Industry,Ins,Trade Type,Price,Qty,Owned,ΔOwn,Value
0,2021-04-29 21:14:49,2021-04-27,TMCI,"Treace Medical Concepts, Inc.",Surgical & Medical Instruments & Apparatus,2,P - Purchase,$17.00,"+17,600","17,600",%,"+$299,200"
1,2021-04-29 20:02:26,2021-04-29,BANC,"Banc of California, Inc.",National Commercial Banks,2,P - Purchase,$18.30,"+32,800","328,944",+11%,"+$600,355"
2,2021-04-28 16:34:02,2021-04-27,SWIM,"Latham Group, Inc.",Plastics Products,6,P - Purchase,$19.00,"+104,500","3,229,888",+3%,"+$1,985,500"
3,2021-04-27 20:02:37,2021-04-23,HCHC,"Hc2 Holdings, Inc.",Fabricated Structural Metal Products,2,P - Purchase,$3.78,"+2,694,986","21,763,717",+14%,"+$10,197,701"
4,2021-04-27 16:50:49,2021-04-27,PWOD,Penns Woods Bancorp Inc,State Commercial Banks,3,P - Purchase,$23.25,"+5,502","34,234",+19%,"+$127,897"
...,...,...,...,...,...,...,...,...,...,...,...,...
91,2021-02-09 19:40:57,2021-02-05,SOWG,Sow Good Inc.,Crude Petroleum & Natural Gas,5,P - Purchase,$4.00,"+225,000","668,881",+51%,"+$900,000"
92,2021-02-05 17:08:21,2021-02-04,JWSM,Jaws Mustang Acquisition Corp,Blank Checks,2,P - Purchase,$10.00,"+200,000","200,000",%,"+$2,000,000"
93,2021-02-03 17:34:32,2021-02-02,PCB,Pcb Bancorp,State Commercial Banks,2,P - Purchase,$11.57,"+15,002","1,358,959",+1%,"+$173,550"
94,2021-02-02 21:31:03,2021-02-02,BLUA,Blueriver Acquisition Corp.,Blank Checks,3,P - Purchase,$10.00,"+2,175,000","2,175,000",%,"+$21,750,000"


In [ ]:
df.drop(columns=['Ins', 'ΔOwn'],axis =1, inplace=True)
filingTime = []
filingDate = []
for index, row in df.iterrows():
  lst = row['Filing\xa0Date'].split(" ")
  filingDate.append(lst[0])
  filingTime.append(lst[1])
print(filingTime)


['21:14:49', '20:02:26', '16:34:02', '20:02:37', '16:50:49', '16:03:37', '20:32:57', '12:04:33', '16:10:07', '16:05:49', '16:58:58', '16:05:15', '19:32:55', '17:27:57', '16:13:39', '13:31:41', '21:39:26', '13:52:11', '14:45:40', '20:23:39', '16:41:11', '20:21:42', '18:35:25', '16:50:51', '17:52:35', '16:11:43', '21:39:38', '17:32:43', '16:28:48', '18:57:34', '16:52:00', '15:01:42', '09:01:50', '17:17:49', '18:23:36', '16:52:38', '16:43:30', '16:39:54', '16:09:49', '16:06:52', '21:59:24', '17:30:35', '17:28:08', '16:23:49', '18:30:08', '17:01:20', '11:31:49', '16:53:09', '16:45:58', '16:05:32', '16:05:30', '10:47:01', '16:11:27', '17:01:07', '18:56:09', '17:35:47', '15:44:07', '13:20:21', '20:08:35', '16:28:02', '06:45:08', '16:41:23', '16:33:11', '16:13:37', '18:42:39', '17:02:02', '11:13:53', '08:00:20', '21:01:16', '21:00:40', '17:48:23', '16:17:26', '20:15:45', '16:22:27', '13:18:50', '19:16:47', '13:48:03', '17:25:04', '16:15:26', '16:47:50', '12:00:50', '18:21:13', '16:52:11', '16

In [ ]:
df.drop(columns=['Filing\xa0Date'],axis =1, inplace=True)
df2 = pd.DataFrame.from_dict({"Filing\xa0Date" : filingDate, "Filing\xa0Time" : filingTime})

df = pd.concat([df2, df], axis=1)
df

,Filing Date,Filing Time,Trade Date,Ticker,Company Name,Industry,Trade Type,Price,Qty,Owned,Value
0,2021-04-29,21:14:49,2021-04-27,TMCI,"Treace Medical Concepts, Inc.",Surgical & Medical Instruments & Apparatus,P - Purchase,$17.00,"+17,600","17,600","+$299,200"
1,2021-04-29,20:02:26,2021-04-29,BANC,"Banc of California, Inc.",National Commercial Banks,P - Purchase,$18.30,"+32,800","328,944","+$600,355"
2,2021-04-28,16:34:02,2021-04-27,SWIM,"Latham Group, Inc.",Plastics Products,P - Purchase,$19.00,"+104,500","3,229,888","+$1,985,500"
3,2021-04-27,20:02:37,2021-04-23,HCHC,"Hc2 Holdings, Inc.",Fabricated Structural Metal Products,P - Purchase,$3.78,"+2,694,986","21,763,717","+$10,197,701"
4,2021-04-27,16:50:49,2021-04-27,PWOD,Penns Woods Bancorp Inc,State Commercial Banks,P - Purchase,$23.25,"+5,502","34,234","+$127,897"
...,...,...,...,...,...,...,...,...,...,...,...
91,2021-02-09,19:40:57,2021-02-05,SOWG,Sow Good Inc.,Crude Petroleum & Natural Gas,P - Purchase,$4.00,"+225,000","668,881","+$900,000"
92,2021-02-05,17:08:21,2021-02-04,JWSM,Jaws Mustang Acquisition Corp,Blank Checks,P - Purchase,$10.00,"+200,000","200,000","+$2,000,000"
93,2021-02-03,17:34:32,2021-02-02,PCB,Pcb Bancorp,State Commercial Banks,P - Purchase,$11.57,"+15,002","1,358,959","+$173,550"
94,2021-02-02,21:31:03,2021-02-02,BLUA,Blueriver Acquisition Corp.,Blank Checks,P - Purchase,$10.00,"+2,175,000","2,175,000","+$21,750,000"
